In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pickle
with open('cropdata', 'rb') as f:
    data = pickle.load(f)
    
for key in data:
    print(key)
    
input_size = data['happy'].shape[1]
data_count = data['happy'].shape[0]
print("Data count = " + str(data_count))
emotion_count = len(data)
hidden_size = 200
batch_size = 100

happy
sad
neutral
Data count = 400


In [2]:
I = tf.placeholder(tf.float32, shape=(None, emotion_count, 1, input_size), name='I')
print(I.shape)

W = tf.Variable(tf.random_uniform((input_size, hidden_size)), name = 'W') / input_size
V = tf.Variable(tf.random_uniform((emotion_count, hidden_size, input_size)), name = 'V') / hidden_size
H = tf.einsum('ijkl,lm->ijkm', I, W)
print(H.shape)
O = tf.einsum('ijkl,jlm->ijkm', H, V)
print(O.shape)

(?, 3, 1, 7000)
(?, 3, 1, 200)
(?, 3, 1, 7000)


In [3]:
loss = tf.reduce_mean(tf.square(O - I))
for i in range(batch_size):
    loss += tf.reduce_mean(tf.square(H[i][0] - H[i][1]))
    loss += tf.reduce_mean(tf.square(H[i][1] - H[i][2]))
    loss += tf.reduce_mean(tf.square(H[i][2] - H[i][0]))


In [4]:
train = tf.train.AdamOptimizer(0.05).minimize(loss)

In [5]:
#
# Preparing data.
newdata = np.concatenate((data['happy'], data['sad'], data['neutral']), axis = 1)
print(newdata.shape)
newdata = newdata.reshape(400, 3, 1, 7000)


# newdata = newdata[:batch_size][:][:][:]
print(newdata.shape)

assert np.all(data['happy'][0] == newdata[0][0])

(400, 21000)
(400, 3, 1, 7000)


In [6]:
def createDataRange(start, end):
    return newdata[start:end][:][:][:]

In [35]:
saver = tf.train.Saver()

WW = None
VW = None

TRAINING_STEPS = 500
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer()) 
    
    # loading network from file.
    try:
        saver.restore(sess, "./model4.ckpt")
        print("Loaded successfully.")
    except Exception as e:
        print('Cant.')
    
    # Actual training.
    for step in range(TRAINING_STEPS):
        for i in range(0, data_count, batch_size):
            loss_val = sess.run([train, loss], feed_dict={I:createDataRange(i, i+batch_size)})[1]
        if step % 30 == 0:
            print(str(step) + " " + str(loss_val))
    
    # Getting the actual values.
    WW = W.eval()
    VW = V.eval()
    print(tf.trainable_variables())
    
    # Saving the model file.
    save_path = saver.save(sess, './model4.ckpt')
    print('Model saved in file: %s ' % save_path)

Loaded successfully.
0 28.615
30 28.4452
60 28.2613
90 28.0699
120 27.9101
150 27.7782
180 27.5564
210 27.3939
240 27.2403
270 27.07
300 26.882
330 26.7278
360 26.5688
390 26.3982
420 26.2117
450 26.0737
480 25.9232
[<tensorflow.python.ops.variables.Variable object at 0x000002E1D13CA0B8>, <tensorflow.python.ops.variables.Variable object at 0x000002E1D6DBAD30>]
Model saved in file: ./model4.ckpt 


In [8]:
print(WW.shape)
print(VW.shape)

(7000, 200)
(3, 200, 7000)


In [9]:
#
#   Now, we recon the output vectors to listen to sound!
#

import librosa
# import numpy as np
from IPython.lib.display import Audio

def invlogamplitude(S):
    """librosa.logamplitude is actually 10_log10, so invert that."""
    return 10.0**(S/10.0)

def recon(mfccs, n_mfcc):
    
    sr = 22050
#     mfccs.reshape((1, input_size))
    mfccs = mfccs.reshape((n_mfcc, input_size // n_mfcc), order='F')
#     print(mfccs.shape)
    
#     n_mfcc = mfccs.shape[0]
    n_mel = 128
    dctm = librosa.filters.dct(n_mfcc, n_mel)
    n_fft = 2048
    mel_basis = librosa.filters.mel(sr, n_fft)

    bin_scaling = 1.0/np.maximum(0.0005, np.sum(np.dot(mel_basis.T, mel_basis),
    axis=0))

    recon_stft = bin_scaling[:, np.newaxis] * np.dot(mel_basis.T,
    invlogamplitude(np.dot(dctm.T, mfccs)))

    excitation = np.random.randn(mfccs.shape[1] * 510)
    E = librosa.stft(excitation)
    recon = librosa.istft(E/np.abs(E)*np.sqrt(recon_stft))

#     print(mfccs.shape)
    return Audio(recon, rate=sr)

In [29]:
# print(data['happy'][0].shape)
testinp = np.array([data['neutral'][20]])
print(testinp.shape)

recon(testinp, 200)

(1, 7000)


In [27]:
hh = np.matmul(testinp, WW)
print(hh.shape)

oo = np.matmul(hh, VW[1])
print(oo.shape)

print(np.max(oo))

recon(oo * 2 , 200)

(1, 200)
(1, 7000)
149.877401106
